---
#### 작업형 1-1.
1. 대륙별 맥주소비량의 평균을 계산하고, 평균이 가장 큰 대륙을 찾아라
2. 위 대륙에서 맥주소비량이 5번째로 많은 국가의 맥주 소비량을 구하라
---

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/drinks.csv")
display(df.shape, df.head())

(193, 6)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [12]:
g = df.groupby('continent')['beer_servings'].mean().reset_index()
g1 = g.sort_values('beer_servings', ascending=False)
print('1.평균이 가장 큰 대륙 = ', g1['continent'].iloc[0])

1.평균이 가장 큰 대륙 =  Europe


In [17]:
df_new = df[df['continent']=='Europe']
c = df_new.sort_values('beer_servings', ascending=False)
print('2.맥주소비량이 5번째로 많은 국가의 맥주 소비량 = ', c['beer_servings'].iloc[4])

2.맥주소비량이 5번째로 많은 국가의 맥주 소비량 =  313


---
#### 작업형 1-2.
1. 관광객 비율이 두번째로 높은 국가의 사업 방문객 수를 a라고 정의하라
2. 관광이 두번째로 높은 국가의 공무 방문객 수를 b라고 정의하라
3. a+b를 구하라
- 방문객 합계 = 관광 + 공무 + 사업 + 기타
- 관광객 비율 = 관광 / 방문객 합계
---

In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/tourist.csv")
display(df.shape, df.head())

(100, 5)

,나라,관광,공무,사업,기타
0,국가1,1184,270,380,55
1,국가2,1059,184,267,86
2,국가3,1129,168,261,50
3,국가4,692,106,214,125
4,국가5,1335,296,296,84


In [27]:
df['관광객비율'] = df['관광'] / (df['관광']+df['공무']+df['사업']+df['기타'])
r1 = df.sort_values('관광객비율', ascending=False)
a = r1.iloc[1,3] #203
r2 = df.sort_values('관광', ascending=False)
b = r2.iloc[1,2] #238
print('정답 = ', a+b)

정답 =  441


---
#### 작업형 1-3.
1. 주어진 데이터에서 co와 nmhc 컬럼을 각각 Min-Max 스케일링 하라
2. 스케일링된 두 컬럼의 표준편차를 각각 구하라
3. co 컬럼의 표준편차에서 nmhc 컬럼의 표준편차를 뺀 값을 구하라 (소수 3자리 반올림)
---

In [28]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/chem.csv")
display(df.shape, df.head())

(100, 4)

,sample,co,nmhc,etc
0,샘플1,79,54,31
1,샘플2,84,57,58
2,샘플3,109,74,113
3,샘플4,15,77,21
4,샘플5,65,77,115


In [33]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['co'] = scaler.fit_transform(df[['co']])
df['nmhc'] = scaler.fit_transform(df[['nmhc']])

s1 = df['co'].std()
s2 = df['nmhc'].std()
print('2.표준편차 = ', s1, s2)
print('3.표준편차의 차이 = ', round(s1-s2, 3))

2.표준편차 =  0.2856516497116944 0.3030617020578397
3.표준편차의 차이 =  -0.017


---
#### 작업형 2-1. 통신사에서 고객에게청구될 총 금액을 예측하라
- 타겟: totalChanges
- 평가: MAE
- 제출: result.csv (pred)
---

In [55]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn_test.csv")
display(train.shape, test.shape, train.head(2), test.head(2))

########### EDA ############################################
# 범주형 다수, 결측치 없음
# train/test 불일치 카테고리 없음, 카테고리 종류 많지 않은 편
############################################################

# 1.타겟분리, id삭제
y = train['TotalCharges']
train = train.drop(['customerID', 'TotalCharges'], axis=1)
test = test.drop('customerID', axis=1)

# 2.범주형 원-핫 인코딩 (수치형의 스케일링은 skip)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 3.검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42)

# 4.model, eval
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
model = RandomForestRegressor(random_state=120)
model.fit(X_train, y_train)

model_pred = model.predict(X_valid)
score = mean_absolute_error(y_valid, model_pred)
print('MAE = ', score)

(4116, 19)

(1764, 18)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,CUST0454,Male,0,No,No,7,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Electronic check,163.45
1,CUST1145,Female,1,No,Yes,53,No,No phone service,Fiber optic,Yes,Yes,No,Yes,Yes,Yes,Month-to-month,No,Credit card,4140.89


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,CUST0769,Male,1,No,Yes,47,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,No,Yes,Month-to-month,Yes,Credit card
1,CUST0675,Male,1,Yes,Yes,12,No,No phone service,Fiber optic,No,Yes,No,Yes,Yes,No,One year,No,Mailed check


MAE =  884.7593588673141


In [59]:
# linear model
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
model2.fit(X_train, y_train)

model2_pred = model2.predict(X_valid)
score = mean_absolute_error(y_valid, model2_pred)
print('MAE_2 = ', score)

from lightgbm import LGBMRegressor
model3 = LGBMRegressor()
model3.fit(X_train, y_train)

model3_pred = model3.predict(X_valid)
score = mean_absolute_error(y_valid, model3_pred)
print('MAE_3 = ', score)

#submit
pred = model2.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv('result.csv', index=False)

MAE_2 =  861.9871651671168
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000333 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 157
[LightGBM] [Info] Number of data points in the train set: 3292, number of used features: 43
[LightGBM] [Info] Start training from score 2572.861462
MAE_3 =  892.3870791621368


---
#### 작업형 3-1. 로지스틱 회귀 (종속변수 : Churn)
1. 주어진 데이터에서 로지스틱 회귀분석을 수행해 유의확률 0.05 이상인 유의하지 않은 독립변수의 갯수를 구하라
2. 유의확률 0.05 미만인 유의한 변수만을 사용해 다시 로지스틱 회귀분석을 시행하고, 유의한 회귀계수들의 합을 구하라 (절편 포함, 반올림하여 소수 3자리)
3. 위에서 수행한 회귀식에서 DataUsage 변수가 5만큼 증가할 때 오즈비를 구하라 (반올림, 소수 3자리)
---

In [78]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn.csv")
display(df.shape, df.head(2))

(1000, 11)

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,51,1,0,1.0,0,303.7,83,72.5,9.3,3.8
1,1,61,1,0,-0.2,2,194.8,89,66.3,10.9,13.3


In [81]:
from statsmodels.formula.api import ols, logit
model = logit('Churn ~ AccountWeeks+ContractRenewal+DataPlan+DataUsage+CustServCalls+DayMins+DayCalls+MonthlyCharge+OverageFee+RoamMins', data=df).fit()
print(model.summary())

print('1.유의하지 않은 독립변수의 갯수 = ', 8)

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      989
Method:                           MLE   Df Model:                           10
Date:                Mon, 15 Sep 2025   Pseudo R-squ.:                 0.02367
Time:                        16:12:48   Log-Likelihood:                -393.60
converged:                       True   LL-Null:                       -403.14
Covariance Type:            nonrobust   LLR p-value:                   0.03924
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.2146      0.913     -2.425      0.015      -4.005      -0.424
AccountWeeks  

In [87]:
model = logit('Churn ~ DataUsage+DayMins', data=df).fit()
print(model.summary())

print('2.유의한 회귀계수들의 합 = ', round((-1.0395 + -0.1697 + -0.0039),3))
print('3.DataUsage 변수가 5만큼 증가할 때 오즈비 = ', round(np.exp(-0.1697 * 5),3))

Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Mon, 15 Sep 2025   Pseudo R-squ.:                 0.01375
Time:                        16:17:33   Log-Likelihood:                -397.60
converged:                       True   LL-Null:                       -403.14
Covariance Type:            nonrobust   LLR p-value:                  0.003908
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0395      0.303     -3.434      0.001      -1.633      -0.446
DataUsage     -0.1697      0.

---
#### 작업형 3-2. 다중회귀
1. 종속변수(PIQ)와 독립변수들로 다중선형 회귀분석을 수행하고, 유의확률이 가장 작은 변수의 회귀계수 값을 구하라(반올림, 소수3자리)
2. 위 모델의 결정계수 값을 구하라 (반올림, 소수 2자리)
3. 뇌 크기(Brain)가 90, 키 70, 몸무게 150일때의 PIQ를 예측하라 (반올림, 정수출력)
---

In [60]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/piq.csv")
display(df.shape, df.head())

(50, 4)

,PIQ,Brain,Height,Weight
0,132,85.78,62.5,127
1,96,86.54,68.0,135
2,84,90.49,66.3,134
3,134,79.06,62.0,122
4,86,88.91,70.0,180


In [65]:
from statsmodels.formula.api import ols, logit
model = ols('PIQ ~ Brain + Height + Weight' ,data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     9.024
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           8.25e-05
Time:                        16:02:51   Log-Likelihood:                -217.27
No. Observations:                  50   AIC:                             442.5
Df Residuals:                      46   BIC:                             450.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.5114     50.564      1.474      0.1

In [73]:
print('1.유의확률이 가장 작은 변수의 회귀계수 = ', round(2.3431, 3))
print('2.결정계수 = ', round(0.370, 2))

new_data = pd.DataFrame({
    'Brain': [90], 'Height':[70], 'Weight':[150]
})
pred = model.predict(new_data)
print('3.PIQ 예측 = ', round(pred.iloc[0]))


1.유의확률이 가장 작은 변수의 회귀계수 =  2.343
2.결정계수 =  0.37
3.PIQ 예측 =  106
